## Work
1. 請改變 reduce_lr 的 patience 和 factor 並比較不同設定下，對訓練/驗證集的影響
2. 請將 optimizer 換成 Adam、RMSprop 搭配 reduce_lr 並比較訓練結果

In [1]:
import os
import keras

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Using TensorFlow backend.


In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
from keras.layers import BatchNormalization
def build_mlp(input_shape, output_units=10, num_neurons=[256, 256, 256]):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(input_layer)
            x = BatchNormalization()(x)
        else:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(x)
            x = BatchNormalization()(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [6]:
## 超參數設定
LEARNING_RATE = 1e-3
EPOCHS = 50
BATCH_SIZE = 1024
MOMENTUM = 0.95

In [7]:
# 載入 Callbacks
from keras.callbacks import ReduceLROnPlateau

"""
建立實驗組合
"""
optimizer_set = [keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=0.95),
                 keras.optimizers.Adam(lr=LEARNING_RATE),
                 keras.optimizers.RMSprop(lr=LEARNING_RATE)]
reduce_lr_factor = [0.99, 0.5, 0.1]
redice_lr_patient = [3,5,10]

In [ ]:
import itertools

"""
以迴圈方式遍歷組合來訓練模型
"""
results = {}
for i, (optim, reduce_factor, reduce_patient) in enumerate(itertools.product(optimizer_set, reduce_lr_factor, redice_lr_patient)):
    print("Numbers of exp: %i, reduce_factor: %.2f, reduce_patient: %i" % (i, reduce_factor, reduce_patient))
    model = build_mlp(input_shape=x_train.shape[1:])
    model.summary()
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optim)
    
    reduce_lr = ReduceLROnPlateau(factor=reduce_factor, 
                              min_lr=1e-12, 
                              monitor='val_loss', 
                              patience=reduce_patient, 
                              verbose=1)
    
    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test), 
              shuffle=True,
              callbacks=[reduce_lr]
             )

    # Collect results
    exp_name_tag = ("exp-%s" % (i))
    results[exp_name_tag] = {'train-loss': model.history.history["loss"],
                             'valid-loss': model.history.history["val_loss"],
                             'train-acc': model.history.history["acc"],
                             'valid-acc': model.history.history["val_acc"]}

W0730 17:15:38.837701 17528 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0730 17:15:38.840693 17528 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



Numbers of exp: 0, reduce_factor: 0.99, reduce_patient: 3


W0730 17:15:39.134822 17528 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0730 17:15:39.861374 17528 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0730 17:15:42.307654 17528 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0730 17:15:42.315634 17528 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 256)               786688    
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               65792     
_________________________________________________________________
batch_normalization_2 (Batch (None, 256)               1024      
_________________________________________________________________
hidden_layer3 (Dense)        (None, 256)               65792     
_________________________________________________________________
batch_normalization_3 (Batch (None, 256)               1024      
__________

W0730 17:15:42.726614 17528 deprecation.py:323] From C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


...

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 53139s 1s/step - loss: 3.0249 - acc: 0.1016 - val_loss: 2.9538 - val_acc: 0.1153
Epoch 2/50
50000/50000 [==============================] - 4s 73us/step - loss: 2.9437 - acc: 0.1129 - val_loss: 2.8937 - val_acc: 0.1245
Epoch 3/50
50000/50000 [==============================] - 4s 72us/step - loss: 2.8932 - acc: 0.1214 - val_loss: 2.8518 - val_acc: 0.1301
Epoch 4/50
50000/50000 [==============================] - 4s 71us/step - loss: 2.8526 - acc: 0.1300 - val_loss: 2.8190 - val_acc: 0.1344
Epoch 5/50
50000/50000 [==============================] - 4s 71us/step - loss: 2.8204 - acc: 0.1351 - val_loss: 2.7916 - val_acc: 0.1388
Epoch 6/50
50000/50000 [==============================] - 4s 71us/step - loss: 2.7930 - acc: 0.1396 - val_loss: 2.7675 - val_acc: 0.1431
Epoch 7/50
50000/50000 [==============================] - 4s 72us/step - loss: 2.7684 - acc: 0.1442 - val_loss: 2.7459 - val_a

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 6s 129us/step - loss: 2.9930 - acc: 0.1081 - val_loss: 2.9215 - val_acc: 0.1152
Epoch 2/50
50000/50000 [==============================] - 4s 73us/step - loss: 2.9021 - acc: 0.1194 - val_loss: 2.8554 - val_acc: 0.1267
Epoch 3/50
50000/50000 [==============================] - 4s 73us/step - loss: 2.8458 - acc: 0.1275 - val_loss: 2.8102 - val_acc: 0.1357
Epoch 4/50
50000/50000 [==============================] - 4s 74us/step - loss: 2.8037 - acc: 0.1335 - val_loss: 2.7743 - val_acc: 0.1420
Epoch 5/50
50000/50000 [==============================] - 4s 75us/step - loss: 2.7675 - acc: 0.1403 - val_loss: 2.7443 - val_acc: 0.1479
Epoch 6/50
50000/50000 [==============================] - 4s 74us/step - loss: 2.7379 - acc: 0.1467 - val_loss: 2.7181 - val_acc: 0.1528
Epoch 7/50
50000/50000 [==============================] - 4s 75us/step - loss: 2.7119 - acc: 0.1505 - val_loss: 2.6950 - val_ac

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 10s 201us/step - loss: 3.2220 - acc: 0.0876 - val_loss: 3.1157 - val_acc: 0.0984
Epoch 2/50
50000/50000 [==============================] - 4s 74us/step - loss: 3.1088 - acc: 0.0957 - val_loss: 3.0299 - val_acc: 0.1057
Epoch 3/50
50000/50000 [==============================] - 4s 73us/step - loss: 3.0367 - acc: 0.1025 - val_loss: 2.9697 - val_acc: 0.1135
Epoch 4/50
50000/50000 [==============================] - 4s 73us/step - loss: 2.9808 - acc: 0.1089 - val_loss: 2.9224 - val_acc: 0.1180
Epoch 5/50
50000/50000 [==============================] - 4s 74us/step - loss: 2.9371 - acc: 0.1139 - val_loss: 2.8842 - val_acc: 0.1240
Epoch 6/50
50000/50000 [==============================] - 4s 75us/step - loss: 2.8985 - acc: 0.1178 - val_loss: 2.8515 - val_acc: 0.1274
Epoch 7/50
50000/50000 [==============================] - 4s 74us/step - loss: 2.8661 - acc: 0.1234 - val_loss: 2.8224 - val_a

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 7s 132us/step - loss: 1.8579 - acc: 0.3625 - val_loss: 3.3008 - val_acc: 0.2148
Epoch 2/50
50000/50000 [==============================] - 4s 76us/step - loss: 1.5446 - acc: 0.4533 - val_loss: 2.4080 - val_acc: 0.3078
Epoch 3/50
50000/50000 [==============================] - 4s 72us/step - loss: 1.4221 - acc: 0.4965 - val_loss: 2.2981 - val_acc: 0.2934
Epoch 4/50
50000/50000 [==============================] - 4s 72us/step - loss: 1.3434 - acc: 0.5243 - val_loss: 1.9681 - val_acc: 0.3403
Epoch 5/50
50000/50000 [==============================] - 4s 73us/step - loss: 1.2796 - acc: 0.5485 - val_loss: 2.2790 - val_acc: 0.3108
Epoch 6/50
50000/50000 [==============================] - 4s 74us/step - loss: 1.2134 - acc: 0.5718 - val_loss: 1.8652 - val_acc: 0.3717
Epoch 7/50
50000/50000 [==============================] - 4s 74us/step - loss: 1.1633 - acc: 0.5884 - val_loss: 1.6701 - val_ac

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 6s 126us/step - loss: 1.8340 - acc: 0.3658 - val_loss: 2.5706 - val_acc: 0.2456
Epoch 2/50
50000/50000 [==============================] - 4s 72us/step - loss: 1.5368 - acc: 0.4575 - val_loss: 3.0542 - val_acc: 0.2082
Epoch 3/50
50000/50000 [==============================] - 4s 73us/step - loss: 1.4159 - acc: 0.4994 - val_loss: 1.7752 - val_acc: 0.3842
Epoch 4/50
50000/50000 [==============================] - 4s 74us/step - loss: 1.3399 - acc: 0.5257 - val_loss: 2.5612 - val_acc: 0.2882
Epoch 5/50
50000/50000 [==============================] - 4s 74us/step - loss: 1.2753 - acc: 0.5538 - val_loss: 1.9402 - val_acc: 0.3538
Epoch 6/50
50000/50000 [==============================] - 4s 72us/step - loss: 1.2185 - acc: 0.5694 - val_loss: 1.9604 - val_acc: 0.3315
Epoch 7/50
50000/50000 [==============================] - 4s 72us/step - loss: 1.1755 - acc: 0.5854 - val_loss: 1.6959 - val_ac

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as mplcm
import matplotlib.colors as colors
%matplotlib inline
NUM_COLORS = len(results.keys())

cm = plt.get_cmap('gist_rainbow')
cNorm  = colors.Normalize(vmin=0, vmax=NUM_COLORS-1)
scalarMap = mplcm.ScalarMappable(norm=cNorm, cmap=cm)
color_bar = [scalarMap.to_rgba(i) for i in range(NUM_COLORS)]

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-loss'])),results[cond]['train-loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-loss'])),results[cond]['valid-loss'], '--', label=cond, color=color_bar[i])
plt.title("Loss")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-acc'])),results[cond]['train-acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-acc'])),results[cond]['valid-acc'], '--', label=cond, color=color_bar[i])
plt.title("Accuracy")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()